In [106]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [108]:
from langchain_community.utilities import SQLDatabase

database_url = os.getenv('DB_URL')

db = SQLDatabase.from_uri(database_url)
print(db.dialect)
print(db.get_usable_table_names())
print(db.get_table_info())
db.run("SELECT * FROM iron_ore_blocks LIMIT 10;")

postgresql
['iron_ore_blocks']

CREATE TABLE iron_ore_blocks (
	"sl._no" BIGINT, 
	state TEXT, 
	block_name TEXT, 
	greenfield_or_brownfield TEXT, 
	status_of_blocks TEXT, 
	type_of_block TEXT, 
	mineral TEXT, 
	name_of_bidder TEXT, 
	lease_area DOUBLE PRECISION, 
	reserve_mt TEXT, 
	fpo DOUBLE PRECISION, 
	date_of_auction DATE, 
	date_of_loi_issuance DATE, 
	fy_expiry TEXT
)

/*
3 rows from iron_ore_blocks table:
sl._no	state	block_name	greenfield_or_brownfield	status_of_blocks	type_of_block	mineral	name_of_bidder	lease_area	reserve_mt	fpo	date_of_auction	date_of_loi_issuance	fy_expiry
1	Andhra Pradesh	Addankivaripalem	Greenfield	Deed Executed and Exploration in Progress	CL	Iron Ore	JSW Steel Ltd.	None	None	14.4	2023-07-28	2023-05-09	2024-25
2	Maharashtra	Ajgaon Block	Greenfield	Deed Executed and Exploration in Progress	CL	Iron Ore	JSW Steel Ltd.	8.4	None	25.0	2022-03-31	None	2023-24
3	Orissa	Badampahar Iron Ore Block	Brownfield	Operationalised	ML	Iron Ore	GM Iron & Steel Company Ltd	

"[(1, 'Andhra Pradesh', 'Addankivaripalem', 'Greenfield', 'Deed Executed and Exploration in Progress', 'CL', 'Iron Ore', 'JSW Steel Ltd.', None, None, 14.4, datetime.date(2023, 7, 28), datetime.date(2023, 5, 9), '2024-25'), (2, 'Maharashtra', 'Ajgaon Block', 'Greenfield', 'Deed Executed and Exploration in Progress', 'CL', 'Iron Ore', 'JSW Steel Ltd.', 8.4, None, 25.0, datetime.date(2022, 3, 31), None, '2023-24'), (3, 'Orissa', 'Badampahar Iron Ore Block', 'Brownfield', 'Operationalised', 'ML', 'Iron Ore', 'GM Iron & Steel Company Ltd', 129.61, '4.484', 95.15, datetime.date(2020, 7, 1), None, '2025-26'), (4, 'Rajasthan', 'Bagawas', 'Brownfield', 'In process of revocation', 'ML', 'Iron Ore', 'Subhash Lohiya, Propo. M/s Bharat Coal Traders, Punjab', 5.9266, '0.58', 452.0, datetime.date(2022, 7, 29), datetime.date(2023, 4, 4), '2028-29'), (5, 'Chhattisgarh', 'Bailadila Deposit 01A Iron Ore Block', 'Greenfield', 'Pending with bidder - SOP/NOC', 'CL', 'Iron Ore', 'M/s ArcelorMittal\\n Nippon

In [109]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-5-mini-2025-08-07'
)

In [110]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x16958a990>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x16958a990>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x16958a990>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [111]:
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

In [48]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [49]:
question = "which state has the most blocks auctioned?"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

which state has the most blocks auctioned?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_8JgDaptiAN0xDolGJLmZAD6l)
 Call ID: call_8JgDaptiAN0xDolGJLmZAD6l
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

iron_ore_blocks
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_nxzbbPK875k2IIxWBs7QWprf)
 Call ID: call_nxzbbPK875k2IIxWBs7QWprf
  Args:
    table_names: iron_ore_blocks
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE iron_ore_blocks (
	"sl._no" BIGINT, 
	state TEXT, 
	block_name TEXT, 
	greenfield_or_brownfield TEXT, 
	status_of_blocks TEXT, 
	type_of_block TEXT, 
	mineral TEXT, 
	name_of_bidder TEXT, 
	lease_area DOU

In [50]:
question = "Describe the iron ore blocks table"

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Describe the iron ore blocks table
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_GiRoKZAoF8ICGuqDTojfC9rr)
 Call ID: call_GiRoKZAoF8ICGuqDTojfC9rr
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

iron_ore_blocks
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_Ogrrq7duf4C834ihfgVplTR2)
 Call ID: call_Ogrrq7duf4C834ihfgVplTR2
  Args:
    table_names: iron_ore_blocks
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE iron_ore_blocks (
	"sl._no" BIGINT, 
	state TEXT, 
	block_name TEXT, 
	greenfield_or_brownfield TEXT, 
	status_of_blocks TEXT, 
	type_of_block TEXT, 
	mineral TEXT, 
	name_of_bidder TEXT, 
	lease_area DOUBLE PREC

In [ ]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


state = query_as_list(db, "SELECT state FROM iron_ore_blocks")
albums = query_as_list(db, "SELECT block_name FROM iron_ore_blocks")
albums[:5]

In [93]:
connection_string = os.getenv('DB_URL')

In [90]:
def get_connection():
    try:
        db = SQLDatabase.from_uri(connection_string)
        return db
    except Exception as e:
        st.error(f'Connection with DB failed {e}')

In [103]:
from langchain_core.documents import Document
def get_pronouns(table_name):
    try:
        db = get_connection()
        query = f"""
                SELECT 
                    column_name, 
                    data_type, 
                    is_nullable,
                    column_default,
                    (SELECT pg_catalog.col_description(c.oid, cols.ordinal_position::int)
                    FROM pg_catalog.pg_class c
                    WHERE c.oid = (SELECT ('"' || cols.table_name || '"')::regclass::oid)
                        AND c.relname = cols.table_name) AS column_comment
                FROM information_schema.columns cols
                WHERE table_name = '{table_name}'
        """
        result_str = db.run(query)
        columns = ast.literal_eval(result_str)
        arr_pronouns = []
        for col in columns:
            if col[1] == 'text':
                col_nouns = (query_as_list(db, f"SELECT {col[0]} FROM iron_ore_blocks"))
                col_unique_nouns = set(col_nouns)
                docs = [Document(page_content=i, metadata={"table": table_name, "column_name": col[0], "column_datatype": col[1]}) for i in col_unique_nouns]
                arr_pronouns+=docs
            # arr_pronouns += query_as_list(db, f"SELECT {col[0]} FROM iron_ore_blocks")
        return arr_pronouns
    except Exception as e:
        st.error(f"Error getting metadata: {e}")
        return []
    finally:
        db._engine.dispose()

In [ ]:
def get_metadata(table_name):
    try:
        query = f"""
                SELECT 
                    column_name, 
                    data_type, 
                    is_nullable,
                    column_default,
                    (SELECT pg_catalog.col_description(c.oid, cols.ordinal_position::int)
                    FROM pg_catalog.pg_class c
                    WHERE c.oid = (SELECT ('"' || cols.table_name || '"')::regclass::oid)
                        AND c.relname = cols.table_name) AS column_comment
                FROM information_schema.columns cols
                WHERE table_name = '{table_name}'
        """
        result_str = db.run(query)
        columns = ast.literal_eval(result_str)
        metadata = [
            f"Field: {col[0]}, Type: {col[1]}, Null: {'YES' if col[2]=='YES' else 'NO'}, "
            
            f"Default: {col[3]}, Comment: {col[4] or ''}"
            for col in columns
        ]
        return metadata
    except Exception as e:
        st.error(f"Error getting metadata: {e}")
        return []

In [82]:
get_metadata("iron_ore_blocks")

['Field: lease_area, Type: double precision, Null: YES, Default: None, Comment: Total lease area of the iron ore block in hectares. Represents the geographical extent of the block allocated for mining.',
 'Field: fpo, Type: double precision, Null: YES, Default: None, Comment: Final price offered (FPO) or winning premium for the auctioned iron ore block, expressed as a percentage. Represents the bid at which the block was awarded to the successful bidder.',
 'Field: date_of_auction, Type: date, Null: YES, Default: None, Comment: The date on which the auction for the iron ore block was conducted. Stored in DATE format. Indian financial calendar convention is that the financial year runs from 1st April to 31st March of the next year.',
 'Field: date_of_loi_issuance, Type: date, Null: YES, Default: None, Comment: The date on which the Letter of Intent (LOI) was issued to the successful bidder. Stored in DATE format. The Indian financial calendar runs from 1st April to 31st March.',
 'Field

In [84]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large'
)


In [85]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [115]:
from langchain.agents.agent_toolkits import create_retriever_tool
import streamlit as st

arr_pronouns = get_pronouns("iron_ore_blocks")

_ = vector_store.add_documents(arr_pronouns)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
description = (
    "The user might make spelling mistake on his input"
    "So consider the user input as an approximate spelling"
    "and try to match with retriever documents provided"
)
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [116]:
retriever_tool.invoke("What is the state name Andra Pradesh")

'Andhra Pradesh\n\nAndhra Pradesh\n\nAndhra Pradesh\n\nAddankivaripalem\n\nAddankivaripalem'

In [ ]:
# Add to system message
suffix = (
    "If you need to filter on a proper noun like a Name, you must ALWAYS first look up "
    "the filter value using the 'search_proper_nouns' tool! Do not try to "
    "guess at the proper name - use this function to find similar ones."
)

system = f"{system_message}\n\n{suffix}"

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, prompt=system)